<a href="https://colab.research.google.com/github/lucasfabs/cours-deeplearning-dv/blob/main/temp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext tensorboard

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
import matplotlib.pyplot as plt
import torch.nn as nn

In [ ]:
import tensorflow as tf
import datetime

In [2]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    # transform=Normalize(),
    # transform=Compose();
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.49MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.3MB/s]


In [3]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.max_pool = nn.MaxPool2d(2)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(1600, 128)
      self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Bloc de convolution 1
        x = self.conv1(x)
        x = F.relu(x)
        x = self.max_pool(x)
        x = self.dropout1(x)

        # Bloc de convolution 2
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool(x)
        x = self.dropout2(x)

        # Aplatissement et couches fully connected
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

# Création du modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
print(model)

# test
random_data = torch.rand((1, 1, 28, 28)).to(device)
result = model(random_data)
print (result)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
tensor([[-0.0365,  0.0024, -0.0858,  0.0618,  0.0124, -0.0128, -0.0864, -0.0722,
          0.0779,  0.0480]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader):
        x, y = batch_value
        x, y = x.to(device), y.to(device)

        # forward pass
        pred = model(x)
        loss = loss_fn(pred, y)

        # backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) + len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0 # sur python associe 1 valeur a deux variables d'un coup
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302293 [   65/60000]
loss: 2.287597 [  165/60000]
loss: 2.291364 [  265/60000]
loss: 2.294849 [  365/60000]
loss: 2.284345 [  465/60000]
loss: 2.267599 [  565/60000]
loss: 2.289262 [  665/60000]
loss: 2.286120 [  765/60000]
loss: 2.274354 [  865/60000]
loss: 2.258306 [  965/60000]
Test Error: 
 Accuracy: 30.0%, Avg loss: 2.266297 

Epoch 2
-------------------------------
loss: 2.279625 [   65/60000]
loss: 2.262983 [  165/60000]
loss: 2.277275 [  265/60000]
loss: 2.250545 [  365/60000]
loss: 2.248358 [  465/60000]
loss: 2.253272 [  565/60000]
loss: 2.241154 [  665/60000]
loss: 2.212111 [  765/60000]
loss: 2.207157 [  865/60000]
loss: 2.191216 [  965/60000]
Test Error: 
 Accuracy: 50.5%, Avg loss: 2.179571 

Epoch 3
-------------------------------
loss: 2.188349 [   65/60000]
loss: 2.195105 [  165/60000]
loss: 2.142713 [  265/60000]
loss: 2.150964 [  365/60000]
loss: 2.144271 [  465/60000]
loss: 2.060053 [  565/60000]
loss: 2.032702 [  665/